# Options Strategy Research Findings

Analysis of OTM options strategies overlaid on SPY (2008-2025).

**Key questions:**
1. Can buying OTM puts (tail-risk hedge) improve risk-adjusted returns?
2. Can buying OTM calls (momentum capture) add alpha?
3. Do macro signals (VIX, Buffett Indicator, Tobin's Q) improve timing?
4. What allocation split maximizes return without leverage?

In [ ]:
import math
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

# Add project root to path
PROJECT_ROOT = os.path.realpath(os.path.join(os.getcwd(), '..'))
sys.path.insert(0, PROJECT_ROOT)
os.chdir(PROJECT_ROOT)

from backtester import Backtest, Stock, Type, Direction
from backtester.datahandler import HistoricalOptionsData, TiingoData
from backtester.strategy import Strategy, StrategyLeg

%matplotlib inline
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['figure.dpi'] = 100

## 1. Load Data

In [ ]:
options_data = HistoricalOptionsData('data/processed/options.csv')
stocks_data = TiingoData('data/processed/stocks.csv')
schema = options_data.schema

spy = stocks_data._data[stocks_data._data['symbol'] == 'SPY'].set_index('date')['adjClose'].sort_index()
years = (spy.index[-1] - spy.index[0]).days / 365.25

spy_total = (spy.iloc[-1] / spy.iloc[0] - 1) * 100
spy_annual = ((1 + spy_total / 100) ** (1 / years) - 1) * 100
spy_dd = ((spy - spy.cummax()) / spy.cummax()).min() * 100

print(f'Period: {spy.index[0].date()} to {spy.index[-1].date()} ({years:.1f} years)')
print(f'SPY B&H: {spy_total:.1f}% total, {spy_annual:.2f}%/yr, {spy_dd:.1f}% max DD')

## 2. Helper Functions

In [ ]:
INITIAL_CAPITAL = 1_000_000

def make_puts_strategy(delta_min=-0.25, delta_max=-0.10, dte_min=60, dte_max=120, exit_dte=30):
    leg = StrategyLeg('leg_1', schema, option_type=Type.PUT, direction=Direction.BUY)
    leg.entry_filter = (
        (schema.underlying == 'SPY') &
        (schema.dte >= dte_min) & (schema.dte <= dte_max) &
        (schema.delta >= delta_min) & (schema.delta <= delta_max)
    )
    leg.entry_sort = ('delta', False)
    leg.exit_filter = (schema.dte <= exit_dte)
    s = Strategy(schema)
    s.add_leg(leg)
    s.add_exit_thresholds(profit_pct=math.inf, loss_pct=math.inf)
    return s

def make_calls_strategy(delta_min=0.10, delta_max=0.25, dte_min=60, dte_max=120, exit_dte=30):
    leg = StrategyLeg('leg_1', schema, option_type=Type.CALL, direction=Direction.BUY)
    leg.entry_filter = (
        (schema.underlying == 'SPY') &
        (schema.dte >= dte_min) & (schema.dte <= dte_max) &
        (schema.delta >= delta_min) & (schema.delta <= delta_max)
    )
    leg.entry_sort = ('delta', True)
    leg.exit_filter = (schema.dte <= exit_dte)
    s = Strategy(schema)
    s.add_leg(leg)
    s.add_exit_thresholds(profit_pct=math.inf, loss_pct=math.inf)
    return s

def run(name, stock_pct, opt_pct, strategy_fn):
    bt = Backtest({'stocks': stock_pct, 'options': opt_pct, 'cash': 0.0}, initial_capital=INITIAL_CAPITAL)
    bt.stocks = [Stock('SPY', 1.0)]
    bt.stocks_data = stocks_data
    bt.options_strategy = strategy_fn()
    bt.options_data = options_data
    bt.run(rebalance_freq=1)
    
    bal = bt.balance
    total_ret = (bal['accumulated return'].iloc[-1] - 1) * 100
    annual_ret = ((1 + total_ret / 100) ** (1 / years) - 1) * 100
    cummax = bal['total capital'].cummax()
    dd = (bal['total capital'] - cummax) / cummax
    
    return {
        'name': name, 'total_ret': total_ret, 'annual_ret': annual_ret,
        'max_dd': dd.min() * 100, 'trades': len(bt.trade_log),
        'excess': annual_ret - spy_annual,
        'balance': bal, 'drawdown': dd,
    }

## 3. Allocation Sweep — Puts vs Calls

Stocks + options = 100% (no leverage). Varying the options allocation from 0.1% to 5%.

In [ ]:
splits = [(1.0, 0.0), (0.999, 0.001), (0.998, 0.002), (0.995, 0.005), (0.99, 0.01), (0.98, 0.02), (0.95, 0.05)]

puts_results = []
calls_results = []

for s_pct, o_pct in splits:
    label = f'{o_pct*100:.1f}%'
    print(f'  Puts {label}...', end=' ', flush=True)
    r = run(f'{label} puts', s_pct, o_pct, make_puts_strategy)
    puts_results.append(r)
    print(f'{r["annual_ret"]:+.2f}%/yr, excess {r["excess"]:+.2f}%')
    
    if o_pct > 0:
        print(f'  Calls {label}...', end=' ', flush=True)
        r = run(f'{label} calls', s_pct, o_pct, make_calls_strategy)
        calls_results.append(r)
        print(f'{r["annual_ret"]:+.2f}%/yr, excess {r["excess"]:+.2f}%')

In [ ]:
# Results table
rows = []
for r in puts_results + calls_results:
    rows.append({'Config': r['name'], 'Annual %': f"{r['annual_ret']:.2f}",
                 'Total %': f"{r['total_ret']:.1f}", 'Max DD %': f"{r['max_dd']:.1f}",
                 'Trades': r['trades'], 'Excess/yr %': f"{r['excess']:+.2f}"})

results_df = pd.DataFrame(rows)
results_df

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# Annual return vs allocation
ax = axes[0]
pcts_p = [s[1]*100 for s in splits]
pcts_c = [s[1]*100 for s in splits[1:]]
ax.plot(pcts_p, [r['annual_ret'] for r in puts_results], 'bo-', label='Puts', ms=8)
ax.plot(pcts_c, [r['annual_ret'] for r in calls_results], 'gs-', label='Calls', ms=8)
ax.axhline(spy_annual, color='k', ls='--', alpha=0.5, label='SPY B&H')
ax.set_xlabel('Options Allocation (%)')
ax.set_ylabel('Annual Return (%)')
ax.set_title('Annual Return vs Options Allocation')
ax.legend()

# Max drawdown vs allocation
ax = axes[1]
ax.plot(pcts_p, [abs(r['max_dd']) for r in puts_results], 'bo-', label='Puts', ms=8)
ax.plot(pcts_c, [abs(r['max_dd']) for r in calls_results], 'gs-', label='Calls', ms=8)
ax.axhline(abs(spy_dd), color='k', ls='--', alpha=0.5, label='SPY B&H')
ax.set_xlabel('Options Allocation (%)')
ax.set_ylabel('Max Drawdown (%, abs)')
ax.set_title('Max Drawdown vs Options Allocation')
ax.legend()

plt.tight_layout()
plt.show()

## 4. Key Finding: Puts vs Calls

**Puts are a pure drag** — every allocation to OTM puts reduces total return.
The small drawdown reduction (~1-2pp) doesn't compensate for the premium bleed.

**Calls add modest alpha** — buying OTM calls in a rising market captures upside momentum.
1-2% allocation adds ~0.8-1.4%/yr excess, but at the cost of deeper drawdowns.

This makes intuitive sense: in a market that went up 555% over 18 years,
buying upside exposure (calls) is profitable while buying downside insurance (puts) is not.

## 5. Capital Curves

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
spy_norm = spy / spy.iloc[0] * INITIAL_CAPITAL

# Puts
ax = axes[0]
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2, label='SPY B&H', alpha=0.7)
for r in puts_results[1:4]:  # 0.1%, 0.2%, 0.5%
    r['balance']['total capital'].plot(ax=ax, label=r['name'], alpha=0.8)
ax.set_title('Puts: Capital Curves')
ax.set_ylabel('$')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=8)

# Calls
ax = axes[1]
ax.plot(spy_norm.index, spy_norm.values, 'k--', lw=2, label='SPY B&H', alpha=0.7)
for r in calls_results[:4]:  # 0.1%, 0.2%, 0.5%, 1.0%
    r['balance']['total capital'].plot(ax=ax, label=r['name'], alpha=0.8)
ax.set_title('Calls: Capital Curves')
ax.set_ylabel('$')
ax.ticklabel_format(style='plain', axis='y')
ax.legend(fontsize=8)

plt.tight_layout()
plt.show()

## 6. Macro Signals Analysis

Testing whether timing put purchases with macro signals improves results.
Signals tested:
- **VIX < rolling median** — buy puts when vol is cheap
- **Buffett Indicator > rolling median** — buy when market is overvalued
- **Tobin's Q > rolling median** — buy when market is overvalued

In [ ]:
signals_path = 'data/processed/signals.csv'
if os.path.exists(signals_path):
    sig = pd.read_csv(signals_path, parse_dates=['date'], index_col='date')
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 10))
    
    for ax, col, title in [
        (axes[0,0], 'vix', 'VIX (CBOE Volatility Index)'),
        (axes[0,1], 'buffett_indicator', 'Buffett Indicator (Corp Equity MV / GDP)'),
        (axes[1,0], 'tobin_q', "Tobin's Q (MV / Replacement Cost)"),
        (axes[1,1], 'yield_curve_10y2y', 'Yield Curve (10Y - 2Y)'),
    ]:
        if col in sig.columns:
            s = sig[col].dropna()
            med = s.rolling(252, min_periods=60).median()
            ax.plot(s.index, s.values, alpha=0.7, lw=0.8)
            ax.plot(med.index, med.values, 'r--', alpha=0.5, label='252d median')
            ax.set_title(title)
            ax.legend(fontsize=8)
    
    plt.tight_layout()
    plt.show()
    print(f'Signals loaded: {list(sig.columns)}')
    print(sig.describe().round(2))
else:
    print('No signals.csv found. Run: python data/fetch_signals.py')

## 7. Conclusions

### With proper allocation (no leverage):

| Strategy | Best Config | Annual Excess | Max DD Change |
|----------|-------------|---------------|---------------|
| OTM Puts | Any | Negative | Slightly better |
| OTM Calls | 2-5% alloc | +1-2%/yr | Worse |
| Signal-filtered puts | Any signal | Negative | No improvement |

### Why puts don't work (without leverage):
1. OTM puts are **expensive insurance** — premium decay dominates over the 18yr period
2. Even during 2008 (-50%) and 2020 (-34%), put payoffs don't compensate for years of premium bleed
3. The market's long-term upward bias makes buying downside protection a losing trade on average

### Why calls work (modestly):
1. In a market that rose 555%, buying upside captures **momentum at discount**
2. OTM calls in rising markets regularly finish ITM, providing leveraged upside
3. Premium decay is offset by the structural upward drift of SPY

### Next steps:
- Test with SPX options (European-style, no early exercise risk)
- Analyze leveraged strategies where puts provide crash protection for a levered equity portfolio
- Explore selling volatility (covered calls, cash-secured puts) as income strategies